In [20]:
#Import Dependencies
import pandas as pd
import numpy as np
import requests
import time
import os
from pprint import pprint



output_path=os.path.join("Exports")

base_url= "https://microdata.worldbank.org/index.php/api/tables/data/fcv/wld_2021_rtfp_v02_m"

In [21]:
all_data= []
params= {
    'format': 'json',
    'limit': 1000,
    'offset': 0
}
while True:
    response=requests.get(base_url, params=params)
    if response.status_code ==200:
        data=response.json()

        if data and 'data' in data:
            all_data.extend(data['data'])

            if len(data['data'])< params['limit']:
                break
            else:
                params['offset']+= params['limit']
        else:
            break
    else:
        print("Error:", response.status_code)
        break

print(f"Total records fetched: {len(all_data)}")

Total records fetched: 502440


In [22]:
data_df=pd.DataFrame(all_data)
data_df.count()

ISO3                          502440
country                       502440
adm1_name                     502440
adm2_name                     502440
mkt_name                      502440
                               ...  
h_food_price_index            502440
l_food_price_index            502440
c_food_price_index            502440
inflation_food_price_index    502440
trust_food_price_index        502440
Length: 536, dtype: int64

In [23]:
data_df.head()

,ISO3,country,adm1_name,adm2_name,mkt_name,lat,lon,geo_id,DATES,year,...,l_yogurt,c_yogurt,inflation_yogurt,trust_yogurt,o_food_price_index,h_food_price_index,l_food_price_index,c_food_price_index,inflation_food_price_index,trust_food_price_index
0,AFG,Afghanistan,Badakhshan,Argo,Badakhshan,37.04,70.46,gid_370400000704600000,2007-01-01,2007,...,,,,,0.61,0.62,0.6,0.61,,7.7
1,AFG,Afghanistan,Badakhshan,Argo,Badakhshan,37.04,70.46,gid_370400000704600000,2007-02-01,2007,...,,,,,0.61,0.62,0.6,0.6,,7.7
2,AFG,Afghanistan,Badakhshan,Argo,Badakhshan,37.04,70.46,gid_370400000704600000,2007-03-01,2007,...,,,,,0.6,0.61,0.6,0.61,,7.7
3,AFG,Afghanistan,Badakhshan,Argo,Badakhshan,37.04,70.46,gid_370400000704600000,2007-04-01,2007,...,,,,,0.61,0.63,0.61,0.63,,7.7
4,AFG,Afghanistan,Badakhshan,Argo,Badakhshan,37.04,70.46,gid_370400000704600000,2007-05-01,2007,...,,,,,0.64,0.69,0.63,0.69,,7.7


In [24]:
dropping=[col for col in data_df.columns if col.startswith(('h_', 'l_'))]
cleaned_df=data_df.drop(columns=dropping)


print(cleaned_df)

       ISO3      country       adm1_name       adm2_name        mkt_name  \
0       AFG  Afghanistan      Badakhshan            Argo      Badakhshan   
1       AFG  Afghanistan      Badakhshan            Argo      Badakhshan   
2       AFG  Afghanistan      Badakhshan            Argo      Badakhshan   
3       AFG  Afghanistan      Badakhshan            Argo      Badakhshan   
4       AFG  Afghanistan      Badakhshan            Argo      Badakhshan   
...     ...          ...             ...             ...             ...   
502435  YEM  Yemen, Rep.  Market Average  Market Average  Market Average   
502436  YEM  Yemen, Rep.  Market Average  Market Average  Market Average   
502437  YEM  Yemen, Rep.  Market Average  Market Average  Market Average   
502438  YEM  Yemen, Rep.  Market Average  Market Average  Market Average   
502439  YEM  Yemen, Rep.  Market Average  Market Average  Market Average   

          lat    lon                    geo_id       DATES  year  ...  \
0       37.04 

In [25]:

additional_columns=['adm1_name', 'adm2_name','lat','lon','geo_id', 'year','month', 'components', 'start_dense_data', 'last_survey_point']
dropping.extend(additional_columns)
cleaned_df=cleaned_df.drop(columns= additional_columns)
print(cleaned_df)

       ISO3      country        mkt_name       DATES currency  data_coverage  \
0       AFG  Afghanistan      Badakhshan  2007-01-01      AFN          34.73   
1       AFG  Afghanistan      Badakhshan  2007-02-01      AFN          34.73   
2       AFG  Afghanistan      Badakhshan  2007-03-01      AFN          34.73   
3       AFG  Afghanistan      Badakhshan  2007-04-01      AFN          34.73   
4       AFG  Afghanistan      Badakhshan  2007-05-01      AFN          34.73   
...     ...          ...             ...         ...      ...            ...   
502435  YEM  Yemen, Rep.  Market Average  2024-11-01      YER          52.09   
502436  YEM  Yemen, Rep.  Market Average  2024-12-01      YER          52.09   
502437  YEM  Yemen, Rep.  Market Average  2025-01-01      YER          52.09   
502438  YEM  Yemen, Rep.  Market Average  2025-02-01      YER          52.09   
502439  YEM  Yemen, Rep.  Market Average  2025-03-01      YER          52.09   

        data_coverage_recent  index_con

In [26]:
os.makedirs(output_path, exist_ok=True)
cleaned_df.to_csv(os.path.join(output_path,"cleaned.csv"), index_label="ISO3")